# Sentiment Analysis 
### Lexicon based SA analysis
I tried out some things, and in the end you will find the calculation of a sentiment score of a sentence. Did this using the sentiwordnet lexicon, but maybe other lexicons are nice to use too? The stuff here can easily be applied to our liar dataset. 

In [5]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

In [6]:
# Download the sentiwordnet data (only needed once...)
nltk.download('sentiwordnet')

[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /Users/joycedenhertog/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


True

In [7]:
print(list(swn.senti_synsets('slow')))

[SentiSynset('decelerate.v.01'), SentiSynset('slow.v.02'), SentiSynset('slow.v.03'), SentiSynset('slow.a.01'), SentiSynset('slow.a.02'), SentiSynset('dense.s.04'), SentiSynset('slow.a.04'), SentiSynset('boring.s.01'), SentiSynset('dull.s.08'), SentiSynset('slowly.r.01'), SentiSynset('behind.r.03')]


In [18]:
breakdown = swn.senti_synset('breakdown.n.03')
print(breakdown)

<breakdown.n.03: PosScore=0.0 NegScore=0.25>


In [9]:
breakdown.pos_score()

0.0

In [42]:
print(swn.senti_synsets('breakdown'))
for synset in swn.senti_synsets('breakdown'):
    print(synset)

<dislocation.n.02: PosScore=0.0 NegScore=0.0>
<breakdown.n.02: PosScore=0.125 NegScore=0.5>
<breakdown.n.03: PosScore=0.0 NegScore=0.25>
<breakdown.n.04: PosScore=0.0 NegScore=0.0>


In [127]:
def senti_score(word):
    synsets =  swn.senti_synsets(word)
    positive = 0
    negative = 0
    total_synsets = 0
    
    for synset in synsets:
        positive = positive + synset.pos_score()
        negative = negative + synset.neg_score()
        total_synsets += 1 
        
    positive = positive / total_synsets
    negative = negative / total_synsets
    return (positive, negative)

In [128]:
for s in swn.senti_synsets(","):
    print(s)

In [152]:
senti_score('hello')

(0.0, 0.0)

In [181]:
sentence = ["this", "is", "a", "sad", "sentence", "sad", "sad"]

In [182]:
def senti_score_sentence(sentence):
    positive = 0
    negative = 0
    length = 0
    
    for word in sentence:
        if not wn.synsets(word): # if it doesn't have synsets you can't calculate the sentiment score..
            continue
        positive = positive + senti_score(word)[0]
        negative = negative + senti_score(word)[1]
        length += 1
    
    positive = positive / length
    negative = negative / length

    return (positive, negative)

In [183]:
senti_score_sentence(sentence)

(0.028617216117216113, 0.34249084249084244)